<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/exercise_11_top_acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 11: Top-K accuracy

In the lecture, we learned to align embedding spaces, which allowed us to "compute" word translations. We marveled at how well this worked, but did not
really evaluate this properly, even though we do have a nice test set.

In the exercise, your task will be to evaluate the method using the simple "top-k accuracy" metric. This is a simple metric, which measures whether the correct target is among the first K nearest neighbors. In other words for the pair of source-target words $w_{fin}, w_{eng}$ we consider the transfer successful, if $w_{eng}$ is among the K nearest neighbors of the embedding we obtain by transforming $w_{ fin}$ with the matrix $M$. Top K accuracy then is the proportion of successfully transferred pairs, out of all pairs, as a percentage.

This can be achieved by a simple modification of the lecture code below.

In [ ]:
import gensim

In [ ]:
# I found this link in the NLPL repository
# It refers to English model trained on the Gigaword corpus of news
#!wget http://vectors.nlpl.eu/repository/20/12.zip
# And this is Finnish, trained on a web crawl data we created back then: https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-1989
#!wget http://vectors.nlpl.eu/repository/20/42.zip

## Try these if the download above is too slow, I mirrored these:
!wget http://dl.turkunlp.org/TKO_7095_2023/12.zip
!wget http://dl.turkunlp.org/TKO_7095_2023/42.zip


In [ ]:
# Somewhat awkwardly, these are numbered files and both
# .zip files contain "model.bin"
# Let's unzip and rename
# -o means "do not ask, overwrite by default"
!unzip -o 12.zip
!mv model.bin en.bin
!unzip -o 42.zip
!mv model.bin fi.bin




*   Now we can load the embeddings
*   These are huge, but they are sorted by frequency, so we can easily limit ourselves to the top 100,000 words, which will be plenty enough for us
*   This is maybe good to note, now we enter the territory of NLP models which count in the gigabytes in size



In [ ]:
# This is how you load the trained embeddings
# check the documentation
# w2v embeddings are traditionlly distributed in one of two formats: a text form, and a binary form
# The embeddings we downloaded above are in the binary form, so we need to indicate that when loading

from gensim.models import KeyedVectors

wv_emb_en=KeyedVectors.load_word2vec_format("en.bin", limit=100000, binary=True)
wv_emb_fi=KeyedVectors.load_word2vec_format("fi.bin", limit=100000, binary=True)

`KeyedVectors` documentation is here: https://radimrehurek.com/gensim/models/keyedvectors.html

# Cross-lingual transfer

* Among the most impressive demonstrations of word embeddings
* Given a list of word pairs between two languages (source and target), one can induce a mapping matrix $M$ which performs a linear transformation of the source language embeddings onto the target language embeddings
* This basically "aligns" the source language embedding space onto the target language embedding space

This is well worth testing! Let's try!

## Bilingual dictionaries

* There are many sources, for example this one:
* https://github.com/codogogo/xling-eval
* The associated paper is here: https://aclanthology.org/P19-1070/

In [ ]:
# Grab the data
!wget https://raw.githubusercontent.com/codogogo/xling-eval/master/bli_datasets/en-fi/yacle.test.freq.2k.en-fi.tsv
!wget https://raw.githubusercontent.com/codogogo/xling-eval/master/bli_datasets/en-fi/yacle.train.freq.5k.en-fi.tsv


In [ ]:
!cat yacle.test.freq.2k.en-fi.tsv | head -n 10

In [ ]:
pairs_train=[] #These will be pairs of (source,target) i.e. (Finnish, English) words used to induce the matrix M
pairs_test=[]  #same but for testing, so we should make sure there is absolutely no overlap between the train and test data
               #let's do it so that not one word in the test is is seen in any capacity in the training data

import csv

def get_vectors(fname):
    """
    Read the pairs from the file `fname`
    """
    pairs=[]
    with open(fname) as f:
        r = csv.reader(f,delimiter="\t") #the file is a .tsv i.e. tab-separated-values
        for en_word,fi_word in r:
            #I will reverse the order here, go from Finnish as the source, to English as the target
            #That way it will be easier to check how this works using English as the target, which we all understand
            pairs.append((fi_word,en_word))
        return pairs

train_data=get_vectors("yacle.train.freq.5k.en-fi.tsv")
test_data=get_vectors("yacle.test.freq.2k.en-fi.tsv")
print(train_data[:10])
print(len(train_data))
print(test_data[:10])
print(len(test_data))



## Get the embeddings

* Now we have the word pairs
* We need the embeddings, so we can build our S and T matrices
* Not all words will be in our W2V embeddings
* Plus, we want to be 100% sure there is absolutely no overlap between the training and test data
* This means not one word seen in the training data will be in the test data
* The general approach will be to gather the vectors into a list, and then vstack (vertical stack) these to get a 2D array, i.e. a matrix

In [ ]:
import numpy

def build_arrays(pairs,emb1,emb2,avoid=set()):
    """
    `pairs`: pairs of (fi,en) words
    `emb1`: source side (here Finnish) embeddings
    `emb2`: target side (here English) embeddings
    `avoid`: a set of words to avoid/ignore (will be used when building test data, to avoid train data)
    """
    vecs1,vecs2,filtered_pairs=[],[],[]  #vectors for source words, vectors for target words, and the word pairs themselves, i.e. three same-length lists
    for w1,w2 in pairs: #Go over all pairs that we got
        # check if both vectors are available, and none of the words is to be avoided
        if w1 in emb1 and w2 in emb2 and w1 not in avoid and w2 not in avoid:
            #passed!
            vecs1.append(emb1[w1]) #source-side embedding, the KeyedVectors object can be queried as if it was a dictionary, returns the embedding as 1-dim array
            vecs2.append(emb2[w2]) #target-side embeddings
            filtered_pairs.append((w1,w2)) #remember the pair
    #Now we vstack() which turns the lists of embeddings into 2-dim array
    return numpy.vstack(vecs1),numpy.vstack(vecs2),filtered_pairs

# Gather the train data first
array_train_fi,array_train_en,pairs_train=build_arrays(train_data,wv_emb_fi,wv_emb_en)
# Now build the set of all words seen in training, so we can avoid them when building the test set. Note that "|" is set union operator
everything_in_train=set(s for s,t in pairs_train)|set(t for s,t in pairs_train)
# Test data next, avoiding the words from the training data:
array_test_fi,array_test_en,pairs_test=build_arrays(test_data,wv_emb_fi,wv_emb_en,avoid=everything_in_train)

In [ ]:
# Let's be super-sure there absolutely is no overlap of any kind!
print("Overlap between train pairs and test pairs:",len(set(pairs_train) & set(pairs_test))) # & is set intersection operator, intersection between train and test should be empty
src_train=set(src_w for src_w,tgt_w in pairs_train) #train source words
tgt_train=set(tgt_w for src_w,tgt_w in pairs_train) #train target words
src_test=set(src_w for src_w,tgt_w in pairs_test)   #test source words
tgt_test=set(tgt_w for src_w,tgt_w in pairs_test)   #test target words
print("Overlap between train fi words and test fi words:",len(src_train & src_test))
print("Overlap between train en words and test en words:",len(tgt_train & tgt_test))

## Mapping matrix
* Next we need to induce the transformation matrix
* I.e. implement the least-squares methods from the lecture
* Surely GPT4 can write this for us:

In [ ]:
# This code was written by GPT4, but in a bit of a twisted form, so I modified it
# to better correspond to the formulae in the lecture

def learn_transformation_matrix(source, target):
    # Compute the pseudo-inverse of the source matrix
    source_pseudo_inverse = numpy.linalg.pinv(source) # This implements (S^T S)^-1 S^T  needed in the least-squares formula in the lecture slides
    # Compute the transformation matrix M using least squares method
    M = numpy.matmul(source_pseudo_inverse,target)  #...and this multiplies by T from right completing the formula in the slides ... two lines(!)
    return M

# fi -> en matrix
M=learn_transformation_matrix(array_train_fi,array_train_en)

# Ha ha well that was easy

In [ ]:
print("Source (fi) shape",array_train_fi.shape)
print("Target (en) shape",array_train_en.shape)
print("M shape",M.shape)


In [ ]:
# And now we transform the source (Finnish) test embeddings into the English embedding space
# using the matrix M
test_fi_transformed=numpy.matmul(array_test_fi,M) # This corresponds to SM in the lecture slides, i.e. source transformed by M to the target embedding space
print("Transformed shape:",test_fi_transformed.shape)
numpy.square(numpy.subtract(test_fi_transformed, array_test_en)).mean() #This is the mean square error of the actual target, and the transformed source, looks small enough :)

* So now we have the originally Finnish embeddings transformed into the English space
* How can we evaluate?

1. Go over the test word pairs (fi,en)
2. Use the transformed Finnish embedding as a query into the English space
3. List top-N English words which appear near this transformed embedding

Conceptually, we go: "Finnish word -> lookup to Finnish vector -> multiply by M to get English vector -> lookup English words nearby"

In [ ]:
for i,(w1,w2) in enumerate(pairs_test[:10]):
    print(f"{w1} (in English {w2}):")
    nearest_neighbors=wv_emb_en.similar_by_vector(test_fi_transformed[i]) #lookup the nearest
    #nearest_neigbors will be tuples (word,similarity_value)
    eng_words=[w for w, score in nearest_neighbors] #just grab the words
    print(f"   ",", ".join(eng_words)) #...and print then ,-separated
    print()

# It cannot be stressed enough, that none of the words in the test data were seen
# during the induction of the transformation matrix M
# 
# We can observe some direct top-1 hits, and in general we see
# the mapping maps the vector very close to the correct place
# in my view, this is quite impressive :)

# Put your exercise code here

* You can start by copying and repurposing the code above. After all, if you can print the target and the nearest neigbors, you can also count them :)

In [ ]:
########## YOUR CODE HERE #############

######## evaluate for K=1,5,10,50 for example #######
########  below is what I got #########
##
##Top-1 accurracy is 17.82101167315175%
##Top-5 accurracy is 34.007782101167315%
##Top-10 accurracy is 42.4124513618677%
##Top-50 accurracy is 61.08949416342413%